# Supply Network Design

## Problem Description
    1. customer
        - demand for a product
        - 需求可由：
            1. 仓库配送
            2. 工厂配送
    2. 仓库depot
        - 最大库存
    3. 工厂factory
        - 最大可生产量
    4. 运输成本
        - factory->depot
        - depot-> customer
        - factory->customer
    5. 优化方向
        1. 满足客户需求
        2. 减少运输成本


### 简单的例子
1. 工厂信息表

    | Factory | Supply (tons) |
    | --- | --- |
    | Liverpool | 150,000 |
    | Brighton |  200,000 |
2. 仓库信息表

    | Depot | Throughput (tons) |
    | --- | --- |
    | Newcastle | 70,000 |
    | Birmingham | 50,000 |
    | London | 100,000 |
    | Exeter | 40,000 |

3. 客户信息表

    | Customer | Demand (tons) |
    | --- | --- |
    | C1 | 50,000 |
    | C2 | 10,000 |
    | C3 | 40,000 |
    | C4 | 35,000 |
    | C5 | 60,000 |
    | C6 | 20,000 |

4. 运输成本表
    | To | Liverpool | Brighton | Newcastle | Birmingham | London | Exeter |
    | --- | --- | --- | --- | --- | --- | --- |
    | Depots |
    | Newcastle  | 0.5 |   - |
    | Birmingham | 0.5 | 0.3 |
    | London     | 1.0 | 0.5 |
    | Exeter     | 0.2 | 0.2 |
    | Customers |
    | C1 | 1.0 | 2.0 |   - | 1.0 |   - |   - |
    | C2 |   - |   - | 1.5 | 0.5 | 1.5 |   - |
    | C3 | 1.5 |   - | 0.5 | 0.5 | 2.0 | 0.2 |
    | C4 | 2.0 |   - | 1.5 | 1.0 |   - | 1.5 |
    | C5 |   - |   - |   - | 0.5 | 0.5 | 0.5 |
    | C6 | 1.0 |   - | 1.0 |   - | 1.5 | 1.5 |

---
## Model Formulation

### 集合
$ \mathcal{F} := \{\text{factory_1},...,\text{factory_k}\}$

$ \mathcal{D} := \{\text{depot_1},...,\text{depot_m}\}$

$ \mathcal{C} := \{\text{customer_1},...,\text{customer_n}\}$

### 参数
$\text{cost}_{s,t} \in \mathbb{R}^+$: $s$到$t$点的运输成本.

$\text{supply}_f \in \mathbb{R}^+$: 工厂$f$的最大可生产量.

$\text{through}_d \in \mathbb{R}^+$: 仓库$d$的最大可运输量(库存).

$\text{demand}_c \in \mathbb{R}^+$: 客户$c$的需求量.

### Decision Variables
$\text{flow}_{s,t} \in \mathbb{N}^+$: Quantity of goods (in tons) that is shipped from source $s$ to destionation $t$.


### Objective Function

- **Cost**: Minimize total shipping costs.

\begin{equation}
\text{Minimize} \quad Z = \sum_{(s,t) \in \text{Cities} \times \text{Cities}}{\text{cost}_{s,t}*\text{flow}_{s,t}}
\end{equation}

### Constraints

- **Factory output**: Flow of goods from a factory must respect maximum capacity.

\begin{equation}
\sum_{t \in \text{Cities}}{\text{flow}_{f,t}} \leq \text{supply}_{f} \quad \forall f \in \text{Factories}
\end{equation}

- **Customer demand**: Flow of goods must meet customer demand.

\begin{equation}
\sum_{s \in \text{Cities}}{\text{flow}_{s,c}} = \text{demand}_{c} \quad \forall c \in \text{Customers}
\end{equation}

- **Depot flow**: Flow into a depot equals flow out of the depot.

\begin{equation}
\sum_{s \in \text{Cities}}{\text{flow}_{s,d}} = 
\sum_{t \in \text{Cities}}{\text{flow}_{d,t}}
\quad \forall d \in \text{Depots}
\end{equation}

- **Depot capacity**: Flow into a depot must respect depot capacity.

\begin{equation}
\sum_{s \in \text{Cities}}{\text{flow}_{s,d}} \leq \text{through}_{d}
\quad \forall d \in \text{Depots}
\end{equation}

---

In [12]:
import numpy as np
import pandas as pd
import json
import gurobipy as grb

In [3]:
# Create dictionaries to capture factory supply limits, depot throughput limits, and customer demand.
supply = dict({'Liverpool': 150000,
               'Brighton': 200000})

through = dict({'Newcastle': 70000,
                'Birmingham': 50000,
                'London': 100000,
                'Exeter': 40000})

demand = dict({'C1': 50000,
               'C2': 10000,
               'C3': 40000,
               'C4': 35000,
               'C5': 60000,
               'C6': 20000})

# Create a dictionary to capture shipping costs.

arcs, cost = grb.multidict({
    ('Liverpool', 'Newcastle'): 0.5,
    ('Liverpool', 'Birmingham'): 0.5,
    ('Liverpool', 'London'): 1.0,
    ('Liverpool', 'Exeter'): 0.2,
    ('Liverpool', 'C1'): 1.0,
    ('Liverpool', 'C3'): 1.5,
    ('Liverpool', 'C4'): 2.0,
    ('Liverpool', 'C6'): 1.0,
    ('Brighton', 'Birmingham'): 0.3,
    ('Brighton', 'London'): 0.5,
    ('Brighton', 'Exeter'): 0.2,
    ('Brighton', 'C1'): 2.0,
    ('Newcastle', 'C2'): 1.5,
    ('Newcastle', 'C3'): 0.5,
    ('Newcastle', 'C5'): 1.5,
    ('Newcastle', 'C6'): 1.0,
    ('Birmingham', 'C1'): 1.0,
    ('Birmingham', 'C2'): 0.5,
    ('Birmingham', 'C3'): 0.5,
    ('Birmingham', 'C4'): 1.0,
    ('Birmingham', 'C5'): 0.5,
    ('London', 'C2'): 1.5,
    ('London', 'C3'): 2.0,
    ('London', 'C5'): 0.5,
    ('London', 'C6'): 1.5,
    ('Exeter', 'C3'): 0.2,
    ('Exeter', 'C4'): 1.5,
    ('Exeter', 'C5'): 0.5,
    ('Exeter', 'C6'): 1.5
})

In [27]:
def model(supply,demand,through,arcs,cost):
    m = grb.Model('SupplyNetworkDesign')
    flow = m.addVars(arcs, obj=cost, name="flow")
    # 定义约束条件
    ## 1
    factories = supply.keys()
    factory_flow = m.addConstrs((grb.quicksum(flow.select(factory, '*')) <= supply[factory]
                                     for factory in factories), name="factory")
    ## 2
    customers = demand.keys()
    customer_flow = m.addConstrs((grb.quicksum(flow.select('*', customer)) == demand[customer]
                                      for customer in customers), name="customer")
    ## 3
    depots = through.keys()
    depot_flow = m.addConstrs((grb.quicksum(flow.select(depot, '*')) == grb.quicksum(flow.select('*', depot))
                                   for depot in depots), name="depot")
    ## 4
    depot_capacity = m.addConstrs((grb.quicksum(flow.select('*', depot)) <= through[depot]
                                   for depot in depots), name="depot_capacity")
    # 求解
    m.optimize()
    # 结果分析
    product_flow = pd.DataFrame(columns=["From", "To", "Flow"])
    for arc in arcs:
        if flow[arc].x > 1e-6:
            product_flow = product_flow.append({"From": arc[0], "To": arc[1], "Flow": flow[arc].x}, ignore_index=True)  
    product_flow.index=[''] * len(product_flow)
    product_flow


In [29]:
model(supply,demand,through,arcs,cost)

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 16 rows, 29 columns and 65 nonzeros
Model fingerprint: 0x3607c855
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 2e+05]
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 15 rows, 29 columns, 64 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4800000e+05   1.812500e+04   0.000000e+00      0s
       7    1.9850000e+05   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.02 seconds
Optimal objective  1.985000000e+05


,From,To,Flow
,Liverpool,C1,50000.0
,Liverpool,C6,20000.0
,Brighton,Birmingham,50000.0
,Brighton,London,55000.0
,Brighton,Exeter,40000.0
,Birmingham,C2,10000.0
,Birmingham,C4,35000.0
,Birmingham,C5,5000.0
,London,C5,55000.0
,Exeter,C3,40000.0


### 例子
extension:仓库是可选的，决定需要使用哪个仓库，附带一个使用仓库的成本；
优化方向: 1.哪几个仓库是需要使用的? 2.Birmingham是否需要扩大规模? 3.为了满足客户需求，怎么配送?

- 仓库信息表(+2个仓库)

    | Depot | Throughput (tons) |
    | --- | --- |
    | Newcastle | 70,000 |
    | Birmingham | 50,000 |
    | London | 100,000 |
    | Exeter | 40,000 |
    |Bristol |	30,00|
    |Northampton | 25,000|

- 仓库使用成本表(可添加其他仓库的使用成本)

    | Depot | Cost to open |
    | --- | --- |
    | Newcastle | 10,000 |
    | Exeter | 5,000 |
    |Bristol |	12,000|
    |Northampton | 4,000|

- 运输成本表
    | To | Liverpool | Brighton | Newcastle | Birmingham | London | Exeter |
    | --- | --- | --- | --- | --- | --- | --- |
    | Depots |
    | Newcastle  | 0.5 |   - |
    | Birmingham | 0.5 | 0.3 |
    | London     | 1.0 | 0.5 |
    | Exeter     | 0.2 | 0.2 |
    |Bristol	 | 0.6 | 0.4 |
    |Northampton | 0.4 | 0.3 |
    | Customers |
    | C1 | 1.0 | 2.0 |   - | 1.0 |   - |   - | 1.2| - |
    | C2 |   - |   - | 1.5 | 0.5 | 1.5 |   - | 0.6| - |
    | C3 | 1.5 |   - | 0.5 | 0.5 | 2.0 | 0.2 | 0.5| - |
    | C4 | 2.0 |   - | 1.5 | 1.0 |   - | 1.5 | -  |0.5|
    | C5 |   - |   - |   - | 0.5 | 0.5 | 0.5 | 0.3|0.8|
    | C6 | 1.0 |   - | 1.0 |   - | 1.5 | 1.5 | 0.8|0.9|

---

## Model Formulation


### Parameters(added)

$\text{opencost}_d \in \mathbb{R}^+$: 使用仓库$d$的成本.

### Decision Variables

$\text{flow}_{s,t} \in \mathbb{N}^+$: Quantity of goods (in tons) that is shipped from source $s$ to destionation $t$.

$\text{open}_{d} \in [0,1]$: Is depot $d$ open?

$\text{expand} \in [0,1]$: Should Birmingham be expanded?


### Objective Function

- **Cost**: Minimize total shipping costs plus costs of opening depots.

\begin{equation}
\text{Minimize} \quad Z = \sum_{(s,t) \in \text{Cities} \times \text{Cities}}{\text{cost}_{s,t}*\text{flow}_{s,t}} +
                          \sum_{{d} \in \text{Depots}}{\text{opencost}_d*\text{open}_d} +
                          3000 * \text{expand}
\end{equation}

### Constraints

- **Factory output**: Flow of goods from a factory must respect maximum capacity.

\begin{equation}
\sum_{t \in \text{Cities}}{\text{flow}_{f,t}} \leq \text{supply}_{f} \quad \forall f \in \text{Factories}
\end{equation}

- **Customer demand**: Flow of goods must meet customer demand.

\begin{equation}
\sum_{s \in \text{Cities}}{\text{flow}_{s,c}} = \text{demand}_{c} \quad \forall c \in \text{Customers}
\end{equation}

- **Depot flow**: Flow into a depot equals flow out of the depot.

\begin{equation}
\sum_{s \in \text{Cities}}{\text{flow}_{s,d}} = 
\sum_{t \in \text{Cities}}{\text{flow}_{d,t}}
\quad \forall d \in \text{Depots}
\end{equation}

- **Depot capacity (all but Birmingham)**: Flow into a depot must respect depot capacity, and is only allowed if the depot is open.

\begin{equation}
\sum_{s \in \text{Cities}}{\text{flow}_{s,d}} \leq \text{through}_{d} * \text{open}_{d}
\quad \forall d \in \text{Depots} - \text{Birmingham}
\end{equation}

- **Depot capacity (Birmingham)**: Flow into Birmingham must respect depot capacity, which may have been expanded.

\begin{equation}
\sum_{s \in \text{Cities}} \text{flow}_{s,\text{Birmingham}} \leq \text{through}_{\text{Birmingham}} + 20000 * \text{expand}
\end{equation}

- **Open depots**: At most 4 open depots (no choice for Birmingham or London).

\begin{equation}
\sum_{d \in \text{Depots}}{\text{open}_{d}} \leq 4
\end{equation}

\begin{equation}
\text{open}_{\text{Birmingham}} = \text{open}_{\text{London}} = 1
\end{equation}

---

In [32]:
supply = dict({'Liverpool': 150000,
               'Brighton': 200000})

through = dict({'Newcastle': 70000,
                'Birmingham': 50000,
                'London': 100000,
                'Exeter': 40000,
                'Bristol': 30000,
                'Northampton': 25000})

opencost = dict({'Newcastle': 10000,
                 'Birmingham': 0,
                 'London': 0,
                 'Exeter': 5000,
                 'Bristol': 12000,
                 'Northampton': 4000})

demand = dict({'C1': 50000,
               'C2': 10000,
               'C3': 40000,
               'C4': 35000,
               'C5': 60000,
               'C6': 20000})

# Create a dictionary to capture shipping costs.

arcs, cost = grb.multidict({
    ('Liverpool', 'Newcastle'): 0.5,
    ('Liverpool', 'Birmingham'): 0.5,
    ('Liverpool', 'London'): 1.0,
    ('Liverpool', 'Exeter'): 0.2,
    ('Liverpool', 'Bristol'): 0.6,
    ('Liverpool', 'Northampton'): 0.4,
    ('Liverpool', 'C1'): 1.0,
    ('Liverpool', 'C3'): 1.5,
    ('Liverpool', 'C4'): 2.0,
    ('Liverpool', 'C6'): 1.0,
    ('Brighton', 'Birmingham'): 0.3,
    ('Brighton', 'London'): 0.5,
    ('Brighton', 'Exeter'): 0.2,
    ('Brighton', 'Bristol'): 0.4,
    ('Brighton', 'Northampton'): 0.3,
    ('Brighton', 'C1'): 2.0,
    ('Newcastle', 'C2'): 1.5,
    ('Newcastle', 'C3'): 0.5,
    ('Newcastle', 'C5'): 1.5,
    ('Newcastle', 'C6'): 1.0,
    ('Birmingham', 'C1'): 1.0,
    ('Birmingham', 'C2'): 0.5,
    ('Birmingham', 'C3'): 0.5,
    ('Birmingham', 'C4'): 1.0,
    ('Birmingham', 'C5'): 0.5,
    ('London', 'C2'): 1.5,
    ('London', 'C3'): 2.0,
    ('London', 'C5'): 0.5,
    ('London', 'C6'): 1.5,
    ('Exeter', 'C3'): 0.2,
    ('Exeter', 'C4'): 1.5,
    ('Exeter', 'C5'): 0.5,
    ('Exeter', 'C6'): 1.5,
    ('Bristol', 'C1'): 1.2,
    ('Bristol', 'C2'): 0.6,
    ('Bristol', 'C3'): 0.5,
    ('Bristol', 'C5'): 0.3,
    ('Bristol', 'C6'): 0.8,
    ('Northampton', 'C2'): 0.4,
    ('Northampton', 'C4'): 0.5,
    ('Northampton', 'C5'): 0.6,
    ('Northampton', 'C6'): 0.9
})

In [40]:
def model2(supply,through,demand,arcs,cost,opencost):
    model = grb.Model('SupplyNetworkDesign2')
    depots = through.keys()
    flow = model.addVars(arcs, obj=cost, name="flow")
    open = model.addVars(depots, obj=opencost, vtype=grb.GRB.BINARY, name="open")
    expand = model.addVar(obj=3000, vtype=grb.GRB.BINARY, name="expand")
    open['Birmingham'].lb = 1
    open['London'].lb = 1
    model.objcon = -(opencost['Newcastle'] + opencost['Exeter']) # Phrased as 'savings from closing'
    # 定义约束
    ## 1
    factories = supply.keys()
    factory_flow = model.addConstrs((grb.quicksum(flow.select(factory, '*')) <= supply[factory]
                                     for factory in factories), name="factory")
    ## 2
    customers = demand.keys()
    customer_flow = model.addConstrs((grb.quicksum(flow.select('*', customer)) == demand[customer]
                                      for customer in customers), name="customer")
    ## 3
    depot_flow = model.addConstrs((grb.quicksum(flow.select(depot, '*')) == grb.quicksum(flow.select('*', depot))
                               for depot in depots), name="depot")
    ## 4
    all_but_birmingham = list(set(depots) - set(['Birmingham']))
    depot_capacity = model.addConstrs((grb.quicksum(flow.select(depot, '*')) <= through[depot]*open[depot]
                                       for depot in all_but_birmingham), name="depot_capacity")
    ## 5
    birmingham_capacity = model.addConstr(grb.quicksum(flow.select('*', 'Birmingham')) <= through['Birmingham'] +
                                      20000*expand, name="birmingham_capacity")
    ## 6
    depot_count = model.addConstr(open.sum() <= 4)
    # 求解
    model.optimize()
    # 结果分析
    print('List of open depots:', [d for d in depots if open[d].x > 0.5])
    if expand.x > 0.5:
        print('Expand Birmingham')
    product_flow = pd.DataFrame(columns=["From", "To", "Flow"])
    for arc in arcs:
        if flow[arc].x > 1e-6:
            product_flow = product_flow.append({"From": arc[0], "To": arc[1], "Flow": flow[arc].x}, ignore_index=True)  
    product_flow.index=[''] * len(product_flow)
    product_flow

In [41]:
model2(supply,through,demand,arcs,cost,opencost)

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 21 rows, 49 columns and 119 nonzeros
Model fingerprint: 0x46146bf2
Variable types: 42 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [2e-01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 2e+05]
Presolve removed 0 rows and 2 columns
Presolve time: 0.00s
Presolved: 21 rows, 47 columns, 113 nonzeros
Variable types: 42 continuous, 5 integer (5 binary)

Root relaxation: objective 1.740000e+05, 17 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    174000.00000 174000.000  0.00%     -    0s

Explored 0 nodes (17 simplex iterations) in 0.02 seconds
Thread count was 4 (of 4 available processors)

Solution 